# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation():
    # Create copies of global variables
    nhl=nhl_df.copy()
    cities_df=cities.copy()
    
    # Cities dataframe data management
    # Remove spaces from the beginning or ending of names of teams
    cities_df['NHL'] = cities_df['NHL'].str.strip()
    # Remove []
    cities_df['NHL']=cities_df['NHL'].str.replace(r'\[.*\]','')
    cities_df['NHL']=cities_df['NHL'].str.replace(r'—','')
    cities_df['NHL'].replace({'': None}, inplace=True)
    cities_df=cities_df[["Metropolitan area", "Population (2016 est.)[8]", "NHL"]]
    cities_df.columns=["Metropolitan area", "Population", "NHL"]
    #print("Total metropolitan areas:", len(cities_df))
    #print("Null: ", cities_df['NHL'].isna().sum())
    cities_df.dropna(subset=["NHL"], inplace=True)
    cities_df.reset_index(drop=True, inplace=True)
    #print(len(cities_df))
    #print(cities_df['NHL'].unique())
    #display(cities_df)
    
    # nhl dataframe data management
    
    # Retrieve and clean names of teams
    #display(nhl.info())
    nhl=nhl.where(nhl["year"]==2018)
    nhl.dropna(inplace=True)
    # Remove spaces from the beginning or ending of names of teams
    nhl['team'] = nhl['team'].str.strip()
    # Drop rows with division names
    #display(nhl)
    nhl['team']=nhl['team'].str.replace(r'.* Division$','')
    nhl['team'].replace({'': None}, inplace=True)
    nhl.dropna(subset=["team"], inplace=True)
    nhl.reset_index(drop=True, inplace=True)
    # Remove extra asterisks from team names
    nhl['team']=nhl['team'].str.replace(r'\*$','')
    
    # Compare names of teams from nhl dataframe with names from the cities_df
    # and replace conserve only the names in cities_df
    old_names=nhl['team'].tolist()
    teams_no_matched = [] # from cities_df
    new_names = [] # New names of teams for nhl dataframe
    flags = []
    #display(nhl)
    for team in nhl['team']:
        flag = False
        for team2 in cities_df['NHL']:
            if team2 in team:
                new_names.append(team2)
                flag = True
                flags.append(flag)
                break
        if flag==False:
            new_names.append(team)
            flags.append(flag)
    #print(len(new_names))
    #print(len(old_names))
    #print(sum(flags))
    temp = pd.DataFrame()
    temp["Old Names"] = new_names
    temp["New Names"] = old_names
    temp["Change?"] = flags
    temp=temp.where(temp["Change?"]==False).dropna()
    #display(temp)
    # Teams in cities_df that not in new_names
    no_teams = []
    for team in cities_df["NHL"]:
        if team in new_names:
            continue
        no_teams.append(team)
    #display(pd.DataFrame(no_teams))
    
    """
    With the results above we can see that we need to reduce 5 teams, 2 from Los Angels and
    3 from New York
    """
    
    # Rename teams in nhl dataframe
    nhl["team"] = new_names
    nhl["team"] = nhl["team"].replace({"New Jersey Devils": "Rangers Islanders Devils", 
                        "New York Islanders": "Rangers Islanders Devils", 
                        "New York Rangers": "Rangers Islanders Devils", 
                        "Anaheim Ducks": "Kings Ducks", 
                        "Los Angeles Kings": "Kings Ducks"})
    
    # Get only the necessary columns from nhl dataframe
    nhl = nhl[["team", "W", "L"]]
    
    # Change data type
    nhl["W"]=nhl["W"].astype(int)
    nhl["L"]=nhl["L"].astype(int)
    
    # Reduce previous renamed rows to only one
    nhl=nhl.groupby("team").agg("sum")
    #display(nhl)
    nhl.reset_index(inplace=True)
    #display(nhl.info())
    
    # Calculate win/loss ratios
    nhl["win/loss ratio"]=nhl["W"]/(nhl["W"]+nhl["L"])
    
    # Rename column "NHL" to "team"
    cities_df = cities_df.rename(columns={"NHL": "team"})
    
    # Perform inner join
    df=pd.merge(cities_df, nhl, on="team")
    
    # Change data type of the "Population" column from string to int
    df["Population"]=df["Population"].astype(int)
    #display(df)
    
    population_by_region = df["Population"].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = df["win/loss ratio"].tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    #return stats.pearsonr(population_by_region, win_loss_by_region)
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nhl_correlation()

0.012308996455744289

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # Create copies of global variables
    nba=nba_df.copy()
    cities_df=cities.copy()
    
    # Cities dataframe data management
    # Remove spaces from the beginning or ending of names of teams
    cities_df['NBA'] = cities_df['NBA'].str.strip()
    # Remove []
    cities_df['NBA']=cities_df['NBA'].str.replace(r'\[.*\]','')
    cities_df['NBA']=cities_df['NBA'].str.replace(r'—','')
    cities_df['NBA'].replace({'': None}, inplace=True)
    cities_df=cities_df[["Metropolitan area", "Population (2016 est.)[8]", "NBA"]]
    cities_df.columns=["Metropolitan area", "Population", "NBA"]
    cities_df.dropna(subset=["NBA"], inplace=True)
    cities_df.reset_index(drop=True, inplace=True)
    #display(cities_df)
    
    # nba dataframe data management
    
    # Retrieve and clean names of teams
    #display(nba)
    #display(nba.info())
    nba=nba.where(nba["year"]==2018)
    nba.dropna(inplace=True)
    #display(nba)
    # Remove spaces from the beginning or ending of names of teams
    nba['team'] = nba['team'].str.strip()
    # Remove extra asterisks from team names
    nba['team']=nba['team'].str.replace(r'\*','')
    # Remove ()
    nba['team']=nba['team'].str.replace(r'\(.*\)$','')
    # Remove spaces from the beginning or ending of names of teams
    nba['team'] = nba['team'].str.strip()
    
    # Compare names of teams from nba dataframe with names from the cities_df
    # and conserve only the names in cities_df
    old_names=nba['team'].tolist()
    teams_no_matched = [] # from cities_df
    new_names = [] # New names of teams for nba dataframe
    flags = []
    #display(nba)
    for team in nba['team']:
        flag = False
        for team2 in cities_df['NBA']:
            if team2 in team:
                new_names.append(team2)
                flag = True
                flags.append(flag)
                break
        if flag==False:
            new_names.append(team)
            flags.append(flag)
    #print("cities: ", len(cities_df))
    #print(len(new_names))
    #print(len(old_names))
    #print(sum(flags))
    temp = pd.DataFrame()
    temp["Old Names"] = new_names
    temp["New Names"] = old_names
    temp["Change?"] = flags
    temp=temp.where(temp["Change?"]==False).dropna()
    #display(temp)
    # Teams in cities_df that not in new_names
    no_teams = []
    for team in cities_df["NBA"]:
        if team in new_names:
            continue
        no_teams.append(team)
    #display(pd.DataFrame(no_teams))
    
    """
    With the results above we can see that we need to reduce 4 teams, 2 from Los Angels and
    2 from New York
    """
    
    # Rename teams in nba dataframe
    nba["team"] = new_names
    nba["team"] = nba["team"].replace({"New York Knicks": "Knicks Nets", 
                        "Brooklyn Nets": "Knicks Nets", 
                        "Los Angeles Clippers": "Lakers Clippers", 
                        "Los Angeles Lakers": "Lakers Clippers"})
    
    # Get only the necessary columns from nba dataframe
    nba = nba[["team", "W", "L"]]
    
    # Change data type
    nba["W"]=nba["W"].astype(int)
    nba["L"]=nba["L"].astype(int)
    
    # Reduce previous renamed rows to only one
    nba=nba.groupby("team").agg("sum")
    #display(nba)
    nba.reset_index(inplace=True)
    #display(nba.info())
    
    # Calculate win/loss ratios
    nba["win/loss ratio"]=nba["W"]/(nba["W"]+nba["L"])
    
    # Rename column "NBA" to "team"
    cities_df = cities_df.rename(columns={"NBA": "team"})
    
    # Perform inner join
    df=pd.merge(cities_df, nba, on="team")
    
    # Change data type of the "Population" column from string to int
    df["Population"]=df["Population"].astype(int)
    #display(df)
    
    population_by_region = df["Population"].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = df["win/loss ratio"].tolist() # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # Create copies of global variables
    mlb=mlb_df.copy()
    cities_df=cities.copy()
    
    # Cities dataframe data management
    # Remove spaces from the beginning or ending of names of teams
    cities_df['MLB'] = cities_df['MLB'].str.strip()
    # Remove []
    cities_df['MLB']=cities_df['MLB'].str.replace(r'\[.*\]','')
    cities_df['MLB']=cities_df['MLB'].str.replace(r'—','')
    cities_df['MLB'].replace({'': None}, inplace=True)
    cities_df=cities_df[["Metropolitan area", "Population (2016 est.)[8]", "MLB"]]
    cities_df.columns=["Metropolitan area", "Population", "MLB"]
    cities_df.dropna(subset=["MLB"], inplace=True)
    cities_df.reset_index(drop=True, inplace=True)
    #display(cities_df)
    
    # mlb dataframe data management
    
    # Retrieve and clean names of teams
    #display(mlb)
    #display(mlb.info())
    mlb=mlb.where(mlb["year"]==2018)
    mlb.dropna(inplace=True)
    #display(mlb)
    # Remove spaces from the beginning or ending of names of teams
    mlb['team'] = mlb['team'].str.strip()
    #display(mlb)
    
    # Compare names of teams from mlb dataframe with names from the cities_df
    # and conserve only the names in cities_df
    old_names=mlb['team'].tolist()
    teams_no_matched = [] # from cities_df
    new_names = [] # New names of teams for mlb dataframe
    flags = []
    #display(mlb)
    for team in mlb['team']:
        flag = False
        for team2 in cities_df['MLB']:
            if team2 in team:
                new_names.append(team2)
                flag = True
                flags.append(flag)
                break
        if flag==False:
            new_names.append(team)
            flags.append(flag)
    #print("cities: ", len(cities_df))
    #print(len(new_names))
    #print(len(old_names))
    #print(sum(flags))
    temp = pd.DataFrame()
    temp["Old Names"] = new_names
    temp["New Names"] = old_names
    temp["Change?"] = flags
    temp=temp.where(temp["Change?"]==False).dropna()
    #display(temp)
    # Teams in cities_df that not in new_names
    no_teams = []
    for team in cities_df["MLB"]:
        if team in new_names:
            continue
        no_teams.append(team)
    #display(pd.DataFrame(no_teams))
    
    """
    With the results above we can see that we need to reduce 8 teams, 2 from Los Angeles,
    2 from New York, 2 from San Francisco-Oakland and 2 from Chicago
    """
    
    # Rename teams in nba dataframe
    mlb["team"] = new_names
    mlb["team"] = mlb["team"].replace({"New York Yankees": "Yankees Mets", 
                        "New York Mets": "Yankees Mets", 
                        "Los Angeles Angels": "Dodgers Angels", 
                        "Los Angeles Dodgers": "Dodgers Angels", 
                        "Oakland Athletics": "Giants Athletics", 
                        "San Francisco Giants": "Giants Athletics", 
                        "Chicago White Sox": "Cubs White Sox", 
                        "Chicago Cubs": "Cubs White Sox"})
    
    # Get only the necessary columns from mlb dataframe
    mlb = mlb[["team", "W", "L"]]
    
    # Change data type
    #mlb["W"]=mlb["W"].astype(int)
    #mlb["L"]=mlb["L"].astype(int)
    
    # Reduce previous renamed rows to only one
    mlb=mlb.groupby("team").agg("sum")
    #display(mlb)
    mlb.reset_index(inplace=True)
    #display(mlb)
    #display(mlb.info())
    
    # Calculate win/loss ratios
    mlb["win/loss ratio"]=mlb["W"]/(mlb["W"]+mlb["L"])
    
    # Rename column "MLB" to "team"
    cities_df = cities_df.rename(columns={"MLB": "team"})
    
    # Perform inner join
    df=pd.merge(cities_df, mlb, on="team")
    
    # Change data type of the "Population" column from string to int
    df["Population"]=df["Population"].astype(int)
    #display(df)
    
    population_by_region = df["Population"].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = df["win/loss ratio"].tolist() # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # Create copies of global variables
    nfl=nfl_df.copy()
    cities_df=cities.copy()
    
    # Cities dataframe data management
    # Remove spaces from the beginning or ending of names of teams
    cities_df['NFL'] = cities_df['NFL'].str.strip()
    # Remove []
    cities_df['NFL']=cities_df['NFL'].str.replace(r'\[.*\]','')
    cities_df['NFL']=cities_df['NFL'].str.replace(r'—','')
    cities_df['NFL'].replace({'': None}, inplace=True)
    cities_df['NFL'].replace({' ': None}, inplace=True) # In NFL Toronto has ' '
    cities_df=cities_df[["Metropolitan area", "Population (2016 est.)[8]", "NFL"]]
    cities_df.columns=["Metropolitan area", "Population", "NFL"]
    cities_df.dropna(subset=["NFL"], inplace=True)
    cities_df.reset_index(drop=True, inplace=True)
    #print(cities_df["NFL"].unique().tolist())
    #display(cities_df)
    
    # Retrieve and clean names of teams
    #display(nfl)
    # Retrieve and clean names of teams
    #display(nfl.info())
    nfl=nfl.where(nfl["year"]==2018)
    nfl.dropna(inplace=True)
    # Remove spaces from the beginning or ending of names of teams
    nfl['team'] = nfl['team'].str.strip()
    # Drop rows with division names
    #display(nfl)
    nfl['team']=nfl['team'].str.replace(r'.* East$|.* North$|.* South$|.* West$','')
    nfl['team'].replace({'': None}, inplace=True)
    nfl.dropna(subset=["team"], inplace=True)
    nfl.reset_index(drop=True, inplace=True)
    # Remove extra asterisks from team names
    nfl['team']=nfl['team'].str.replace(r'\*$','')
    # Remove extra plus signs from team names
    nfl['team']=nfl['team'].str.replace(r'\+$','')
    #display(nfl)
    
    # Compare names of teams from nfl dataframe with names from the cities_df
    # and conserve only the names in cities_df
    old_names=nfl['team'].tolist()
    teams_no_matched = [] # from cities_df
    new_names = [] # New names of teams for nfl dataframe
    flags = []
    #display(nfl)
    for team in nfl['team']:
        flag = False
        for team2 in cities_df['NFL']:
            if team2 in team:
                new_names.append(team2)
                flag = True
                flags.append(flag)
                break
        if flag==False:
            new_names.append(team)
            flags.append(flag)
    #print("cities: ", len(cities_df))
    #print(len(new_names))
    #print(len(old_names))
    #print(sum(flags))
    temp = pd.DataFrame()
    temp["Old Names"] = new_names
    temp["New Names"] = old_names
    temp["Change?"] = flags
    temp=temp.where(temp["Change?"]==False).dropna()
    #display(temp)
    # Teams in cities_df that not in new_names
    no_teams = []
    for team in cities_df["NFL"]:
        if team in new_names:
            continue
        no_teams.append(team)
    #display(pd.DataFrame(no_teams))
    
    """
    With the results above we can see that we need to reduce 6 teams, 2 from Los Angeles,
    2 from New York and 2 from San Francisco-Oakland
    """
    
    # Rename teams in nba dataframe
    nfl["team"] = new_names
    nfl["team"] = nfl["team"].replace({"New York Jets": "Giants Jets", 
                        "New York Giants": "Giants Jets", 
                        "Los Angeles Chargers": "Rams Chargers", 
                        "Los Angeles Rams": "Rams Chargers", 
                        "Oakland Raiders": "49ers Raiders", 
                        "San Francisco 49ers": "49ers Raiders"})
    
    # Get only the necessary columns from nfl dataframe
    nfl = nfl[["team", "W", "L"]]
    #display(nfl.info())
    
    # Change data type
    nfl["W"]=nfl["W"].astype(int)
    nfl["L"]=nfl["L"].astype(int)
    
    # Reduce previous renamed rows to only one
    nfl=nfl.groupby("team").agg("sum")
    #display(nfl)
    nfl.reset_index(inplace=True)
    #display(nfl)
    #display(nfl.info())
    
    # Calculate win/loss ratios
    nfl["win/loss ratio"]=nfl["W"]/(nfl["W"]+nfl["L"])
    
    # Rename column "NFL" to "team"
    cities_df = cities_df.rename(columns={"NFL": "team"})
    
    # Perform inner join
    df=pd.merge(cities_df, nfl, on="team")
    
    # Change data type of the "Population" column from string to int
    df["Population"]=df["Population"].astype(int)
    #display(df)
    
    population_by_region = df["Population"].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = df["win/loss ratio"].tolist() # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

0.004922112149349414

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [55]:
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def get_nhl():
    # Create copies of global variables
    nhl=nhl_df.copy()
    cities_df=cities.copy()
    
    # Cities dataframe data management
    # Remove spaces from the beginning or ending of names of teams
    cities_df['NHL'] = cities_df['NHL'].str.strip()
    # Remove []
    cities_df['NHL']=cities_df['NHL'].str.replace(r'\[.*\]','')
    cities_df['NHL']=cities_df['NHL'].str.replace(r'—','')
    cities_df['NHL'].replace({'': None}, inplace=True)
    cities_df=cities_df[["Metropolitan area", "Population (2016 est.)[8]", "NHL"]]
    cities_df.columns=["Metropolitan area", "Population", "NHL"]
    #print("Total metropolitan areas:", len(cities_df))
    #print("Null: ", cities_df['NHL'].isna().sum())
    cities_df.dropna(subset=["NHL"], inplace=True)
    cities_df.reset_index(drop=True, inplace=True)
    #print(len(cities_df))
    #print(cities_df['NHL'].unique())
    #display(cities_df)
    
    # nhl dataframe data management
    
    # Retrieve and clean names of teams
    #display(nhl.info())
    nhl=nhl.where(nhl["year"]==2018)
    nhl.dropna(inplace=True)
    # Remove spaces from the beginning or ending of names of teams
    nhl['team'] = nhl['team'].str.strip()
    # Drop rows with division names
    #display(nhl)
    nhl['team']=nhl['team'].str.replace(r'.* Division$','')
    nhl['team'].replace({'': None}, inplace=True)
    nhl.dropna(subset=["team"], inplace=True)
    nhl.reset_index(drop=True, inplace=True)
    # Remove extra asterisks from team names
    nhl['team']=nhl['team'].str.replace(r'\*$','')
    
    # Compare names of teams from nhl dataframe with names from the cities_df
    # and replace conserve only the names in cities_df
    old_names=nhl['team'].tolist()
    teams_no_matched = [] # from cities_df
    new_names = [] # New names of teams for nhl dataframe
    flags = []
    #display(nhl)
    for team in nhl['team']:
        flag = False
        for team2 in cities_df['NHL']:
            if team2 in team:
                new_names.append(team2)
                flag = True
                flags.append(flag)
                break
        if flag==False:
            new_names.append(team)
            flags.append(flag)
    #print(len(new_names))
    #print(len(old_names))
    #print(sum(flags))
    temp = pd.DataFrame()
    temp["Old Names"] = new_names
    temp["New Names"] = old_names
    temp["Change?"] = flags
    temp=temp.where(temp["Change?"]==False).dropna()
    #display(temp)
    # Teams in cities_df that not in new_names
    no_teams = []
    for team in cities_df["NHL"]:
        if team in new_names:
            continue
        no_teams.append(team)
    #display(pd.DataFrame(no_teams))
    
    """
    With the results above we can see that we need to reduce 5 teams, 2 from Los Angels and
    3 from New York
    """
    
    # Rename teams in nhl dataframe
    nhl["team"] = new_names
    nhl["team"] = nhl["team"].replace({"New Jersey Devils": "Rangers Islanders Devils", 
                        "New York Islanders": "Rangers Islanders Devils", 
                        "New York Rangers": "Rangers Islanders Devils", 
                        "Anaheim Ducks": "Kings Ducks", 
                        "Los Angeles Kings": "Kings Ducks"})
    
    # Get only the necessary columns from nhl dataframe
    nhl = nhl[["team", "W", "L"]]
    
    # Change data type
    nhl["W"]=nhl["W"].astype(int)
    nhl["L"]=nhl["L"].astype(int)
    
    # Reduce previous renamed rows to only one
    nhl=nhl.groupby("team").agg("sum")
    #display(nhl)
    nhl.reset_index(inplace=True)
    #display(nhl.info())
    
    # Calculate win/loss ratios
    nhl["win/loss ratio"]=nhl["W"]/(nhl["W"]+nhl["L"])
    
    # Rename column "NHL" to "team"
    cities_df = cities_df.rename(columns={"NHL": "team"})
    
    # Perform inner join
    df=pd.merge(cities_df, nhl, on="team")
    
    # Change data type of the "Population" column from string to int
    df["Population"]=df["Population"].astype(int)
    
    return df

In [56]:
nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def get_nba():
    # Create copies of global variables
    nba=nba_df.copy()
    cities_df=cities.copy()
    
    # Cities dataframe data management
    # Remove spaces from the beginning or ending of names of teams
    cities_df['NBA'] = cities_df['NBA'].str.strip()
    # Remove []
    cities_df['NBA']=cities_df['NBA'].str.replace(r'\[.*\]','')
    cities_df['NBA']=cities_df['NBA'].str.replace(r'—','')
    cities_df['NBA'].replace({'': None}, inplace=True)
    cities_df=cities_df[["Metropolitan area", "Population (2016 est.)[8]", "NBA"]]
    cities_df.columns=["Metropolitan area", "Population", "NBA"]
    cities_df.dropna(subset=["NBA"], inplace=True)
    cities_df.reset_index(drop=True, inplace=True)
    #display(cities_df)
    
    # nba dataframe data management
    
    # Retrieve and clean names of teams
    #display(nba)
    #display(nba.info())
    nba=nba.where(nba["year"]==2018)
    nba.dropna(inplace=True)
    #display(nba)
    # Remove spaces from the beginning or ending of names of teams
    nba['team'] = nba['team'].str.strip()
    # Remove extra asterisks from team names
    nba['team']=nba['team'].str.replace(r'\*','')
    # Remove ()
    nba['team']=nba['team'].str.replace(r'\(.*\)$','')
    # Remove spaces from the beginning or ending of names of teams
    nba['team'] = nba['team'].str.strip()
    # Drop rows with division names
    #display(nba)
    nba['team']=nba['team'].str.replace(r'.* Division$','')
    nba['team'].replace({'': None}, inplace=True)
    nba.dropna(subset=["team"], inplace=True)
    nba.reset_index(drop=True, inplace=True)
    
    # Compare names of teams from nba dataframe with names from the cities_df
    # and conserve only the names in cities_df
    old_names=nba['team'].tolist()
    teams_no_matched = [] # from cities_df
    new_names = [] # New names of teams for nba dataframe
    flags = []
    #display(nba)
    for team in nba['team']:
        flag = False
        for team2 in cities_df['NBA']:
            if team2 in team:
                new_names.append(team2)
                flag = True
                flags.append(flag)
                break
        if flag==False:
            new_names.append(team)
            flags.append(flag)
    #print("cities: ", len(cities_df))
    #print(len(new_names))
    #print(len(old_names))
    #print(sum(flags))
    temp = pd.DataFrame()
    temp["Old Names"] = new_names
    temp["New Names"] = old_names
    temp["Change?"] = flags
    temp=temp.where(temp["Change?"]==False).dropna()
    #display(temp)
    # Teams in cities_df that not in new_names
    no_teams = []
    for team in cities_df["NBA"]:
        if team in new_names:
            continue
        no_teams.append(team)
    #display(pd.DataFrame(no_teams))
    
    """
    With the results above we can see that we need to reduce 4 teams, 2 from Los Angels and
    2 from New York
    """
    
    # Rename teams in nba dataframe
    nba["team"] = new_names
    nba["team"] = nba["team"].replace({"New York Knicks": "Knicks Nets", 
                        "Brooklyn Nets": "Knicks Nets", 
                        "Los Angeles Clippers": "Lakers Clippers", 
                        "Los Angeles Lakers": "Lakers Clippers"})
    
    # Get only the necessary columns from nba dataframe
    nba = nba[["team", "W", "L"]]
    
    # Change data type
    nba["W"]=nba["W"].astype(int)
    nba["L"]=nba["L"].astype(int)
    
    # Reduce previous renamed rows to only one
    nba=nba.groupby("team").agg("sum")
    #display(nba)
    nba.reset_index(inplace=True)
    #display(nba.info())
    
    # Calculate win/loss ratios
    nba["win/loss ratio"]=nba["W"]/(nba["W"]+nba["L"])
    
    # Rename column "NBA" to "team"
    cities_df = cities_df.rename(columns={"NBA": "team"})
    
    # Perform inner join
    df=pd.merge(cities_df, nba, on="team")
    
    # Change data type of the "Population" column from string to int
    df["Population"]=df["Population"].astype(int)

    return df
#get_nba()

In [57]:
mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def get_mlb(): 
    # Create copies of global variables
    mlb=mlb_df.copy()
    cities_df=cities.copy()
    
    # Cities dataframe data management
    # Remove spaces from the beginning or ending of names of teams
    cities_df['MLB'] = cities_df['MLB'].str.strip()
    # Remove []
    cities_df['MLB']=cities_df['MLB'].str.replace(r'\[.*\]','')
    cities_df['MLB']=cities_df['MLB'].str.replace(r'—','')
    cities_df['MLB'].replace({'': None}, inplace=True)
    cities_df=cities_df[["Metropolitan area", "Population (2016 est.)[8]", "MLB"]]
    cities_df.columns=["Metropolitan area", "Population", "MLB"]
    cities_df.dropna(subset=["MLB"], inplace=True)
    cities_df.reset_index(drop=True, inplace=True)
    #display(cities_df)
    
    # mlb dataframe data management
    
    # Retrieve and clean names of teams
    #display(mlb)
    #display(mlb.info())
    mlb=mlb.where(mlb["year"]==2018)
    mlb.dropna(inplace=True)
    #display(mlb)
    # Remove spaces from the beginning or ending of names of teams
    mlb['team'] = mlb['team'].str.strip()
    #display(mlb)
    
    # Compare names of teams from mlb dataframe with names from the cities_df
    # and conserve only the names in cities_df
    old_names=mlb['team'].tolist()
    teams_no_matched = [] # from cities_df
    new_names = [] # New names of teams for mlb dataframe
    flags = []
    #display(mlb)
    for team in mlb['team']:
        flag = False
        for team2 in cities_df['MLB']:
            if team2 in team:
                new_names.append(team2)
                flag = True
                flags.append(flag)
                break
        if flag==False:
            new_names.append(team)
            flags.append(flag)
    #print("cities: ", len(cities_df))
    #print(len(new_names))
    #print(len(old_names))
    #print(sum(flags))
    temp = pd.DataFrame()
    temp["Old Names"] = new_names
    temp["New Names"] = old_names
    temp["Change?"] = flags
    temp=temp.where(temp["Change?"]==False).dropna()
    #display(temp)
    # Teams in cities_df that not in new_names
    no_teams = []
    for team in cities_df["MLB"]:
        if team in new_names:
            continue
        no_teams.append(team)
    #display(pd.DataFrame(no_teams))
    
    """
    With the results above we can see that we need to reduce 8 teams, 2 from Los Angeles,
    2 from New York, 2 from San Francisco-Oakland and 2 from Chicago
    """
    
    # Rename teams in nba dataframe
    mlb["team"] = new_names
    mlb["team"] = mlb["team"].replace({"New York Yankees": "Yankees Mets", 
                        "New York Mets": "Yankees Mets", 
                        "Los Angeles Angels": "Dodgers Angels", 
                        "Los Angeles Dodgers": "Dodgers Angels", 
                        "Oakland Athletics": "Giants Athletics", 
                        "San Francisco Giants": "Giants Athletics", 
                        "Chicago White Sox": "Cubs White Sox", 
                        "Chicago Cubs": "Cubs White Sox"})
    
    # Get only the necessary columns from mlb dataframe
    mlb = mlb[["team", "W", "L"]]
    
    # Change data type
    #mlb["W"]=mlb["W"].astype(int)
    #mlb["L"]=mlb["L"].astype(int)
    
    # Reduce previous renamed rows to only one
    mlb=mlb.groupby("team").agg("sum")
    #display(mlb)
    mlb.reset_index(inplace=True)
    #display(mlb)
    #display(mlb.info())
    
    # Calculate win/loss ratios
    mlb["win/loss ratio"]=mlb["W"]/(mlb["W"]+mlb["L"])
    
    # Rename column "MLB" to "team"
    cities_df = cities_df.rename(columns={"MLB": "team"})
    
    # Perform inner join
    df=pd.merge(cities_df, mlb, on="team")
    
    # Change data type of the "Population" column from string to int
    df["Population"]=df["Population"].astype(int)
    
    return df

In [58]:
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def get_nfl(): 
    # Create copies of global variables
    nfl=nfl_df.copy()
    cities_df=cities.copy()
    
    # Cities dataframe data management
    # Remove spaces from the beginning or ending of names of teams
    cities_df['NFL'] = cities_df['NFL'].str.strip()
    # Remove []
    cities_df['NFL']=cities_df['NFL'].str.replace(r'\[.*\]','')
    cities_df['NFL']=cities_df['NFL'].str.replace(r'—','')
    cities_df['NFL'].replace({'': None}, inplace=True)
    cities_df['NFL'].replace({' ': None}, inplace=True) # In NFL Toronto has ' '
    cities_df=cities_df[["Metropolitan area", "Population (2016 est.)[8]", "NFL"]]
    cities_df.columns=["Metropolitan area", "Population", "NFL"]
    cities_df.dropna(subset=["NFL"], inplace=True)
    cities_df.reset_index(drop=True, inplace=True)
    #print(cities_df["NFL"].unique().tolist())
    #display(cities_df)
    
    # Retrieve and clean names of teams
    #display(nfl)
    # Retrieve and clean names of teams
    #display(nfl.info())
    nfl=nfl.where(nfl["year"]==2018)
    nfl.dropna(inplace=True)
    # Remove spaces from the beginning or ending of names of teams
    nfl['team'] = nfl['team'].str.strip()
    # Drop rows with division names
    #display(nfl)
    nfl['team']=nfl['team'].str.replace(r'.* East$|.* North$|.* South$|.* West$','')
    nfl['team'].replace({'': None}, inplace=True)
    nfl.dropna(subset=["team"], inplace=True)
    nfl.reset_index(drop=True, inplace=True)
    # Remove extra asterisks from team names
    nfl['team']=nfl['team'].str.replace(r'\*$','')
    # Remove extra plus signs from team names
    nfl['team']=nfl['team'].str.replace(r'\+$','')
    #display(nfl)
    
    # Compare names of teams from nfl dataframe with names from the cities_df
    # and conserve only the names in cities_df
    old_names=nfl['team'].tolist()
    teams_no_matched = [] # from cities_df
    new_names = [] # New names of teams for nfl dataframe
    flags = []
    #display(nfl)
    for team in nfl['team']:
        flag = False
        for team2 in cities_df['NFL']:
            if team2 in team:
                new_names.append(team2)
                flag = True
                flags.append(flag)
                break
        if flag==False:
            new_names.append(team)
            flags.append(flag)
    #print("cities: ", len(cities_df))
    #print(len(new_names))
    #print(len(old_names))
    #print(sum(flags))
    temp = pd.DataFrame()
    temp["Old Names"] = new_names
    temp["New Names"] = old_names
    temp["Change?"] = flags
    temp=temp.where(temp["Change?"]==False).dropna()
    #display(temp)
    # Teams in cities_df that not in new_names
    no_teams = []
    for team in cities_df["NFL"]:
        if team in new_names:
            continue
        no_teams.append(team)
    #display(pd.DataFrame(no_teams))
    
    """
    With the results above we can see that we need to reduce 6 teams, 2 from Los Angeles,
    2 from New York and 2 from San Francisco-Oakland
    """
    
    # Rename teams in nba dataframe
    nfl["team"] = new_names
    nfl["team"] = nfl["team"].replace({"New York Jets": "Giants Jets", 
                        "New York Giants": "Giants Jets", 
                        "Los Angeles Chargers": "Rams Chargers", 
                        "Los Angeles Rams": "Rams Chargers", 
                        "Oakland Raiders": "49ers Raiders", 
                        "San Francisco 49ers": "49ers Raiders"})
    
    # Get only the necessary columns from nfl dataframe
    nfl = nfl[["team", "W", "L"]]
    #display(nfl.info())
    
    # Change data type
    nfl["W"]=nfl["W"].astype(int)
    nfl["L"]=nfl["L"].astype(int)
    
    # Reduce previous renamed rows to only one
    nfl=nfl.groupby("team").agg("sum")
    #display(nfl)
    nfl.reset_index(inplace=True)
    #display(nfl)
    #display(nfl.info())
    
    # Calculate win/loss ratios
    nfl["win/loss ratio"]=nfl["W"]/(nfl["W"]+nfl["L"])
    
    # Rename column "NFL" to "team"
    cities_df = cities_df.rename(columns={"NFL": "team"})
    
    # Perform inner join
    df=pd.merge(cities_df, nfl, on="team")
    
    # Change data type of the "Population" column from string to int
    df["Population"]=df["Population"].astype(int)

    return df

In [68]:
def get_df(sport):
    if sport =="NHL":
        return get_nhl()
    elif sport =="NBA":
        return get_nba()
    elif sport =="MLB":
        return get_mlb()
    elif sport =="NFL":
        return get_nfl()
    else:
        print("ERROR!")

In [70]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    df_dict = {}
    
    # Load dataframes
    for sport in sports:
        df_dict[sport] = get_df(sport)
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    # Calculate p_values
    for sport1 in sports:
        for sport2 in sports:
            if sport1!=sport2:
                df=pd.merge(df_dict[sport1], df_dict[sport2], how="inner", on = "Metropolitan area")
                p_values.loc[sport1, sport2] = stats.ttest_rel(df["win/loss ratio"+"_x"], df["win/loss ratio"+"_y"])[1]
    
    #return p_values
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030959,0.802384
NBA,0.941792,NaN,0.022316,0.951046
NHL,0.030959,0.022316,NaN,0.000712
MLB,0.802384,0.951046,0.000712,NaN
